---
title: "Pan-UKB Pleitropy of Diseases v01"
author: "Saikat Banerjee"
format:
  html: default
date: "2023-12-11"
file-modified: "2023-12-08"
abstract: "We look at the pleiotropy of some common diseases from the Pan-UKB data."

---

# Getting Setup

In [1]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
from pymir import mpl_stylesheet
from pymir import mpl_utils
mpl_stylesheet.banskt_presentation(splinecolor = 'black', dpi = 120, colors = 'kelly')
from matplotlib.gridspec import GridSpec

from nnwmf.optimize import IALM
from nnwmf.optimize import FrankWolfe, FrankWolfe_CV
from nnwmf.utils import model_errors as merr

import sys
sys.path.append("../utils/")
import simulate as mpy_simulate

# Load data and results

In [2]:
data_dir = "/gpfs/commons/home/sbanerjee/work/npd/PanUKB/data"
result_dir = "/gpfs/commons/home/sbanerjee/work/npd/PanUKB/nnwmf"
zscore_filename = f"{data_dir}/GWAS_Zscore.tsv"
trait_filename = f"{data_dir}/trait_manifest_TableS6_no_readme.tsv"
zscore_df = pd.read_csv(zscore_filename, sep = '\t')
trait_df = pd.read_csv(trait_filename, sep = '\t')

# remove extra columns from trait_df

colnames = trait_df.columns.tolist()
colnames[0] = "zindex"
trait_df.columns = colnames
trait_df_mod = trait_df.drop(labels = ['coding', 'modifier', 'coding_description', 'filename', 'aws_link'], axis=1)
trait_df_mod['trait_name'] = trait_df_mod['description']
trait_df_mod['trait_name'] = trait_df_mod['trait_name'].fillna(trait_df_mod['phenocode'])

In [3]:
X_nan = np.array(zscore_df.loc[:, zscore_df.columns!='rsid']).T
X_nan_cent = X_nan - np.nanmean(X_nan, axis = 0, keepdims = True)
X_nan_mask = np.isnan(X_nan)
X_cent = np.nan_to_num(X_nan_cent, copy = True, nan = 0.0)

print (f"We have {X_cent.shape[0]} samples (phenotypes) and {X_cent.shape[1]} features (variants)")
print (f"Fraction of Nan entries: {np.sum(X_nan_mask) / np.prod(X_cent.shape):.3f}")

We have 2483 samples (phenotypes) and 51399 features (variants)
Fraction of Nan entries: 0.000


In [4]:
mf_methods = ['ialm', 'nnm_sparse', 'tsvd']

method_prefix = {
    'ialm' : 'ialm_maxiter10000_admm',
    'nnm_sparse' : 'nnm_sparse_maxiter1000'
}

method_names = {
    'tsvd' : 'Raw Data',
    'ialm' : 'RPCA-IALM',
    'nnm'  : 'NNM-FW',
    'nnm_weighted' : 'NNM-Weighted',
    'nnm_sparse' : 'NNM-Sparse-FW',
}

with open (f"{result_dir}/{method_prefix['ialm']}_progress.pkl", 'rb') as handle:
    ialm = pickle.load(handle)
    
with open (f"{result_dir}/{method_prefix['nnm_sparse']}_progress.pkl", 'rb') as handle:
    nnm_sparse = pickle.load(handle)

In [5]:
def get_principal_components(X):
    X_cent = mpy_simulate.do_standardize(X, scale = False)
    X_cent /= np.sqrt(np.prod(X_cent.shape))
    U, S, Vt = np.linalg.svd(X_cent, full_matrices = False)
    pcomps = U @ np.diag(S)
    loadings = Vt.T @ np.diag(S)
    return loadings, pcomps, S

lowrank_X = dict()
loadings  = dict()
pcomps    = dict()
eigenvals = dict()

for m in mf_methods:
    if m != 'tsvd':
        with open (f"{result_dir}/{method_prefix[m]}_lowrank_X.pkl", 'rb') as handle:
            lowrank_X[m] = pickle.load(handle)
lowrank_X['tsvd'] = X_cent.copy()
for m in mf_methods:
    loadings[m], pcomps[m], eigenvals[m] = get_principal_components(lowrank_X[m])

In [6]:
def get_cos2_scores(pcomps):
    ntrait, npcomp = pcomps.shape
    x = np.zeros((ntrait, npcomp))
    std_pcomps = pcomps / np.sqrt(np.var(pcomps[:,:], axis = 0))
    for i in range(ntrait):
        cos2_trait = np.array([np.square(std_pcomps[i, pcidx]) for pcidx in range(npcomp)])
        x[i, :] = cos2_trait / np.sum(cos2_trait)
    return x

def get_contribution_scores(pcomps):
    ntrait, npcomp = pcomps.shape
    x = np.zeros((ntrait, npcomp))
    std_pcomps = pcomps / np.sqrt(np.var(pcomps[:,:], axis = 0)).reshape(1, -1)
    for i in range(npcomp):
        trait_contr = np.array([np.square(std_pcomps[j, i]) for j in range(ntrait)])
        x[:, i] = trait_contr / np.sum(trait_contr)
    return x

npcomp = 100
cos2_scores = dict()
for m in mf_methods:
    cos2_scores[m] = get_cos2_scores(pcomps[m][:,:npcomp])

trait_contributions = dict()
for m in mf_methods:
    trait_contributions[m] = get_contribution_scores(pcomps[m][:,:npcomp])

# Pleiotropy

In the following, we look at the pleiotropy of different diseases reported in the UKB project.
Given a phenotype (focal trait), we find the hidden factor with the highest contribution to the focal trait. 
We then report the other phenotypes which contribute to that hidden factor. 

*Function* `pan_ukb_pleiotropy(trait_index)`

**Input**: 
  - Index of the focal trait as found in the [manifest file](https://docs.google.com/spreadsheets/d/16OqNQf8cSk2szlmQWYlNOj30NHMjojyf).
  - Number of pleiotropic traits to be reported (optional, default 15).

**Output**: 
  - Index of the top hidden factor.
  - Contribution of the hidden factor on the focal trait.
  - List of pleiotropic traits, and their contributions to the hidden factor.
  
**Global Variables**: `trait_df_mod`, `mf_methods`, `cos2_scores`, `trait_contributions`, `method_names`

In [52]:
from IPython.display import Markdown, display

import re
def titlecase(s):
    return re.sub(r"[A-Za-z]+('[A-Za-z]+)?", lambda mo: mo.group(0).capitalize(),s)

def printmd(string, color = None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))
    
def printheader(string):
    headerstr=f"<h2>{string}</h2>"
    display(Markdown(headerstr))

def pan_ukb_pleiotropy(znum, num_trait = 15):
    zindex = znum - 1
    focal_trait_name = trait_df_mod.loc[zindex, 'trait_name']
    printheader(titlecase(focal_trait_name))
    for m in mf_methods:
        printmd (f"**{method_names[m]}**", color = "#c8635a")
        top_factor = np.argsort(cos2_scores[m][zindex,:])[::-1][0]
        printmd(f"Top Factor: {top_factor + 1} (Contribution: {cos2_scores[m][zindex,top_factor]:.4f})")
        top_traits = np.argsort(trait_contributions[m][:,top_factor])[::-1][:num_trait]
        trait_names = trait_df_mod.loc[top_traits]['trait_name'].tolist()
        for i, x in zip(top_traits, trait_names):
            print(f"    {trait_contributions[m][i, top_factor]:.4f}   {x}")
        print(f"")

In [43]:
pan_ukb_pleiotropy(930)

<h2>Rheumatoid Arthritis</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 47 (Contribution: 0.1697)</span>

    0.0236   methotrexate
    0.0236   anti-metabolite
    0.0167   folic acid supplement|Food Supplement
    0.0167   folic acid
    0.0146   Forced expiratory volume in 1-second (FEV1), Best measure
    0.0141   Forced expiratory volume in 1-second (FEV1)
    0.0131   Forced vital capacity (FVC), Best measure
    0.0126   Forced vital capacity (FVC)
    0.0113   anti-inflammatory
    0.0113   sulfasalazine
    0.0104   Rheumatoid arthritis
    0.0097   Neutrophill percentage
    0.0097   Rheumatoid arthritis and other inflammatory polyarthropathies
    0.0095   Corneal resistance factor (left)
    0.0093   Corneal resistance factor (right)



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 88 (Contribution: 0.0664)</span>

    0.0319   Whole body fat-free mass
    0.0251   Leg fat percentage (right)
    0.0194   Hand grip strength (left)
    0.0120   Leg predicted mass (right)
    0.0118   Forced vital capacity (FVC), Best measure
    0.0110   codine/paracetamol
    0.0103   Arm fat-free mass (right)
    0.0099   Mean arterial pressure, automated reading
    0.0096   Trunk fat-free mass
    0.0096   Peak expiratory flow (PEF)
    0.0092   Trunk predicted mass
    0.0090   Basal metabolic rate
    0.0089   Diastolic blood pressure, combined automated + manual reading, adjusted by medication
    0.0071   Arm fat mass (left)
    0.0069   Age completed full time education



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 73 (Contribution: 0.1319)</span>

    0.0213   I35 Nonrheumatic aortic valve disorders
    0.0211   Nonrheumatic aortic valve disorders
    0.0173   Congenital anomalies of great vessels
    0.0157   Cardiac congenital anomalies
    0.0147   Cardiac and circulatory congenital anomalies
    0.0137   Albumin
    0.0128   anti-metabolite
    0.0128   methotrexate
    0.0126   Heart valve disorders
    0.0094   Aortic valve disease
    0.0093   Rheumatoid arthritis
    0.0090   Rheumatoid arthritis and other inflammatory polyarthropathies
    0.0080   Osteoporosis
    0.0080   Osteoporosis NOS
    0.0080   Treatment/medication code



In [47]:
pan_ukb_pleiotropy(420)

<h2>Cancer Of Prostate</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 78 (Contribution: 0.0875)</span>

    0.0272   Contracture of palmar fascia [Dupuytren's disease]
    0.0262   Disorders of muscle, ligament, and fascia
    0.0243   Fasciitis
    0.0186   Malignant neoplasm of bladder
    0.0182   C67 Malignant neoplasm of bladder
    0.0157   Cancer of bladder
    0.0156   M72 Fibroblastic disorders
    0.0131   K76 Other diseases of liver
    0.0130   Other disorders of liver
    0.0127   Sebaceous cyst
    0.0125   Diseases of sebaceous glands
    0.0114   Prostatitis
    0.0112   Chronic liver disease and cirrhosis
    0.0110   L72 Follicular cysts of skin and subcutaneous tissue
    0.0107   Cancer of urinary organs (incl. kidney and bladder)



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 48 (Contribution: 0.0815)</span>

    0.0247   Weight
    0.0190   Whole body water mass
    0.0134   Trunk fat mass
    0.0132   Age at first live birth
    0.0128   Weight
    0.0123   Arm fat-free mass (left)
    0.0119   Overweight, obesity and other hyperalimentation
    0.0112   Body mass index (BMI)
    0.0108   Leg fat percentage (right)
    0.0089   Hip circumference
    0.0088   Leg fat-free mass (right)
    0.0081   Trunk predicted mass
    0.0064   aspirin
    0.0064   Leg fat-free mass (left)
    0.0063   sodium alginate / potassium bicarbonate



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 54 (Contribution: 0.0663)</span>

    0.0277   Albumin
    0.0221   IGF-1
    0.0168   Calcium
    0.0163   Alkaline phosphatase
    0.0114   Corneal resistance factor (left)
    0.0106   Corneal resistance factor (right)
    0.0098   Total protein
    0.0089   Corneal hysteresis (left)
    0.0079   Corneal hysteresis (right)
    0.0065   Phosphate
    0.0065   Home location - east co-ordinate (rounded)
    0.0063   Urate
    0.0062   Home location at assessment - east co-ordinate (rounded)
    0.0060   Other disorders of bladder
    0.0056   Albumin/Globulin ratio



In [72]:
pan_ukb_pleiotropy(410)

<h2>Breast Cancer</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 25 (Contribution: 0.1298)</span>

    0.0221   Breast cancer [female]
    0.0215   Breast cancer
    0.0210   Malignant neoplasm of female breast
    0.0200   C50 Malignant neoplasm of breast
    0.0167   Arthropathy NOS
    0.0167   Other arthropathies
    0.0129   M17 Gonarthrosis [arthrosis of knee]
    0.0110   Osteoarthrosis
    0.0108   Unspecified monoarthritis
    0.0098   Diseases of esophagus
    0.0095   Diaphragmatic hernia
    0.0093   Osteoarthritis; localized
    0.0091   K44 Diaphragmatic hernia
    0.0090   Esophagitis, GERD and related diseases
    0.0088   Malignant neoplasm, other



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 66 (Contribution: 0.0840)</span>

    0.0300   Leg fat-free mass (right)
    0.0273   Whole body fat-free mass
    0.0264   Sitting height
    0.0208   Arm fat-free mass (left)
    0.0128   Impedance of leg (left)
    0.0128   Body mass index (BMI)
    0.0120   Arm predicted mass (right)
    0.0109   Whole body water mass
    0.0098   Type 2 diabetes
    0.0097   Waist circumference
    0.0097   Arm predicted mass (left)
    0.0085   Arm fat-free mass (right)
    0.0081   E66 Obesity
    0.0078   Impedance of leg (right)
    0.0074   lansoprazole



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 48 (Contribution: 0.0812)</span>

    0.0133   Urate
    0.0109   Past tobacco smoking
    0.0108   Breast cancer
    0.0108   Breast cancer [female]
    0.0101   Malignant neoplasm of female breast
    0.0099   C50 Malignant neoplasm of breast
    0.0099   Smoking status, ever vs never
    0.0090   Age first had sexual intercourse
    0.0086   Alkaline phosphatase
    0.0075   Diseases of esophagus
    0.0072   Gamma glutamyltransferase
    0.0071   Aspartate aminotransferase
    0.0070   Esophagitis, GERD and related diseases
    0.0067   Duration to first press of snap-button in each round
    0.0062   Estimated glomerular filtration rate, serum creatinine + cystain C



In [49]:
pan_ukb_pleiotropy(458)

<h2>Diabetes Mellitus</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 19 (Contribution: 0.1403)</span>

    0.0364   Red blood cell (erythrocyte) count
    0.0189   Glycated haemoglobin (HbA1c)
    0.0178   Mean corpuscular volume
    0.0159   Lymphocyte count
    0.0150   Mean sphered cell volume
    0.0146   Haemoglobin concentration
    0.0137   Platelet count
    0.0136   Haematocrit percentage
    0.0132   Mean corpuscular haemoglobin
    0.0128   Type 2 diabetes
    0.0127   Non-albumin protein
    0.0124   E11 Non-insulin-dependent diabetes mellitus
    0.0119   Diabetes mellitus
    0.0108   Platelet crit
    0.0106   Neutrophill percentage



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 57 (Contribution: 0.0559)</span>

    0.0178   Arm fat-free mass (left)
    0.0162   Trunk fat mass
    0.0144   Body mass index (BMI)
    0.0140   codine/paracetamol
    0.0135   Type 2 diabetes
    0.0131   Comparative height size at age 10
    0.0107   tetracycline antibiotic|antibiotic
    0.0095   Impedance of arm (right)
    0.0092   Hip circumference
    0.0092   Diabetes mellitus
    0.0089   Forced expiratory volume in 1-second (FEV1), Best measure
    0.0082   ACE inhibitor|anti-hypertensive
    0.0077   Trunk fat-free mass
    0.0069   trimethoprim
    0.0069   Weight



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 26 (Contribution: 0.1503)</span>

    0.1042   HDL cholesterol
    0.0898   Apolipoprotein A
    0.0473   Glycated haemoglobin (HbA1c)
    0.0273   Albumin/Globulin ratio
    0.0231   Non-albumin protein
    0.0208   Platelet crit
    0.0192   Neutrophill percentage
    0.0175   Neutrophill count
    0.0154   Lymphocyte percentage
    0.0138   IGF-1
    0.0131   Gamma glutamyltransferase
    0.0112   Type 2 diabetes
    0.0111   Diabetes mellitus
    0.0111   E11 Non-insulin-dependent diabetes mellitus
    0.0109   Mean platelet (thrombocyte) volume



In [53]:
pan_ukb_pleiotropy(529)

<h2>Parkinson's Disease</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 97 (Contribution: 0.0888)</span>

    0.0441   Psoriasis
    0.0425   L40 Psoriasis
    0.0403   Psoriasis and related disorders
    0.0187   Cardiac arrest
    0.0184   Cardiac arrest and ventricular fibrillation
    0.0181   I46 Cardiac arrest
    0.0117   K90 Intestinal malabsorption
    0.0114   Intestinal malabsorption (non-celiac)
    0.0109   Obesity
    0.0108   E66 Obesity
    0.0097   Overweight, obesity and other hyperalimentation
    0.0095   Aortic aneurysm
    0.0094   I71 Aortic aneurysm and dissection
    0.0090   Celiac disease
    0.0085   Noninflammatory disorders of ovary, fallopian tube, and broad ligament



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 70 (Contribution: 0.0852)</span>

    0.0262   Trunk fat percentage
    0.0240   Whole body fat-free mass
    0.0207   Leg predicted mass (left)
    0.0201   Arm fat-free mass (left)
    0.0172   Leg fat percentage (left)
    0.0147   Leg fat mass (right)
    0.0105   Leg fat mass (left)
    0.0083   NSAID|non-steroidal anti-inflammatory drug
    0.0079   Usual walking pace
    0.0078   proton pump inhibitor|PPI
    0.0073   Impedance of leg (left)
    0.0071   Estimated glomerular filtration rate, serum creatinine + cystain C
    0.0065   Arm fat mass (right)
    0.0061   Diastolic blood pressure, combined automated + manual reading, adjusted by medication
    0.0059   codine/paracetamol



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 92 (Contribution: 0.1208)</span>

    0.0149   Urinary calculus
    0.0149   N20 Calculus of kidney and ureter
    0.0147   Maximum heart rate during fitness test
    0.0146   ECG, load
    0.0146   Maximum workload during fitness test
    0.0139   Pulse rate, automated reading
    0.0138   ECG, heart rate
    0.0130   ECG, number of stages in a phase
    0.0126   Number of trend entries
    0.0117   ECG, phase time
    0.0110   Calculus of ureter
    0.0102   Processed meat intake
    0.0101   Calculus of kidney
    0.0088   Beef intake
    0.0080   Lamb/mutton intake



In [55]:
pan_ukb_pleiotropy(504)

<h2>Dementias</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 99 (Contribution: 0.1643)</span>

    0.0148   Cardiac arrest
    0.0146   Cardiac arrest and ventricular fibrillation
    0.0144   I46 Cardiac arrest
    0.0130   Other disorders of pancreatic internal secretion
    0.0125   Hypoglycemia
    0.0122   E16 Other disorders of pancreatic internal secretion
    0.0112   Obesity
    0.0111   E66 Obesity
    0.0104   E21 Hyperparathyroidism and other disorders of parathyroid gland
    0.0104   G47 Sleep disorders
    0.0103   Hyperparathyroidism
    0.0101   Sleep disorders
    0.0101   Overweight, obesity and other hyperalimentation
    0.0090   Varicose veins of lower extremity
    0.0088   Disorders of parathyroid gland



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 74 (Contribution: 0.0601)</span>

    0.0203   Number of self-reported non-cancer illnesses
    0.0169   Leg predicted mass (left)
    0.0162   Waist circumference
    0.0160   Hip circumference
    0.0139   Mean arterial pressure, combined automated + manual reading
    0.0136   Arm fat mass (left)
    0.0130   Diastolic blood pressure, combined automated + manual reading
    0.0129   NSAID|non-steroidal anti-inflammatory drug
    0.0129   Arm fat-free mass (left)
    0.0128   Systolic blood pressure, combined automated + manual reading, adjusted by medication
    0.0128   Impedance of arm (left)
    0.0109   Essential hypertension
    0.0106   Body mass index (BMI)
    0.0098   Diastolic blood pressure, combined automated + manual reading, adjusted by medication
    0.0092   Peak expiratory flow (PEF)



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 61 (Contribution: 0.0862)</span>

    0.0324   Heel bone mineral density (BMD)
    0.0322   Heel bone mineral density (BMD) T-score, automated
    0.0322   Heel quantitative ultrasound index (QUI), direct entry
    0.0258   Heel Broadband ultrasound attenuation, direct entry
    0.0139   Urate
    0.0119   Heel bone mineral density (BMD) T-score, automated (left)
    0.0119   Heel quantitative ultrasound index (QUI), direct entry (left)
    0.0119   Heel broadband ultrasound attenuation (left)
    0.0116   Heel bone mineral density (BMD) (left)
    0.0115   Heel bone mineral density (BMD) T-score, automated (right)
    0.0115   Heel quantitative ultrasound index (QUI), direct entry (right)
    0.0115   Heel broadband ultrasound attenuation (right)
    0.0114   Heel bone mineral density (BMD) (right)
    0.0103   Cerebrovascular disease
    0.0102   Creatinine



In [59]:
pan_ukb_pleiotropy(61)

<h2>G47 Sleep Disorders</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 86 (Contribution: 0.1228)</span>

    0.0469   Acquired foot deformities
    0.0468   M20 Acquired deformities of fingers and toes
    0.0290   Hemoptysis
    0.0286   R04 Haemorrhage from respiratory passages
    0.0275   Obesity
    0.0272   E66 Obesity
    0.0272   Hallux valgus (Bunion)
    0.0253   Acquired toe deformities
    0.0248   Abnormal sputum
    0.0247   Overweight, obesity and other hyperalimentation
    0.0123   G47 Sleep disorders
    0.0119   Sleep disorders
    0.0111   Hammer toe (acquired)
    0.0110   Epistaxis or throat hemorrhage
    0.0095   Prostatitis



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 43 (Contribution: 0.0683)</span>

    0.0234   ACE inhibitor|anti-hypertensive
    0.0141   High light scatter reticulocyte count
    0.0133   Usual walking pace
    0.0121   Weight
    0.0119   Leg predicted mass (left)
    0.0109   Trunk fat-free mass
    0.0104   Estimated glomerular filtration rate, serum creatinine + cystain C
    0.0100   Reticulocyte count
    0.0092   Trunk fat percentage
    0.0091   Waist circumference
    0.0081   Age first had sexual intercourse
    0.0077   Seated height
    0.0066   Treatment/medication code
    0.0064   Other arthropathies
    0.0061   Ankle spacing width (right)



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 91 (Contribution: 0.1037)</span>

    0.0136   Aspartate aminotransferase
    0.0125   Platelet distribution width
    0.0112   Alanine aminotransferase
    0.0084   Inflammatory bowel disease and other gastroenteritis and colitis
    0.0074   Spherical power (right)
    0.0070   Genital prolapse
    0.0070   Prolapse of vaginal walls
    0.0069   Spherical power (left)
    0.0068   N81 Female genital prolapse
    0.0068   Open wounds of head; neck; and trunk
    0.0063   C79 Secondary malignant neoplasm of other sites
    0.0061   E87 Other disorders of fluid, electrolyte and acid-base balance
    0.0061   Ulcerative colitis
    0.0061   Electrolyte imbalance
    0.0061   I77 Other disorders of arteries and arterioles



In [61]:
pan_ukb_pleiotropy(59)

<h2>G43 Migraine</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 80 (Contribution: 0.0792)</span>

    0.0312   Appendicitis
    0.0307   Acute appendicitis
    0.0287   Appendiceal conditions
    0.0284   K35 Acute appendicitis
    0.0208   Sebaceous cyst
    0.0203   Diseases of sebaceous glands
    0.0186   L72 Follicular cysts of skin and subcutaneous tissue
    0.0140   Contracture of palmar fascia [Dupuytren's disease]
    0.0134   Disorders of muscle, ligament, and fascia
    0.0133   Iron deficiency anemias, unspecified or not due to blood loss
    0.0133   Iron deficiency anemias
    0.0121   Fasciitis
    0.0118   D50 Iron deficiency anaemia
    0.0115   Home location - north co-ordinate (rounded)
    0.0113   Home location at assessment - north co-ordinate (rounded)



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 46 (Contribution: 0.0576)</span>

    0.0320   Leg predicted mass (right)
    0.0257   Whole body fat-free mass
    0.0243   Leg fat-free mass (right)
    0.0192   Body mass index (BMI)
    0.0167   Leg fat mass (left)
    0.0144   NSAID|non-steroidal anti-inflammatory drug
    0.0120   simvastatin
    0.0112   thiazide diuretic|diuretic
    0.0109   aspirin
    0.0092   lansoprazole
    0.0091   ACE inhibitor|anti-hypertensive
    0.0089   Diastolic blood pressure, automated reading
    0.0085   Non-cancer illness code, self-reported
    0.0070   bendroflumethiazide
    0.0070   amoxicillin



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 56 (Contribution: 0.0733)</span>

    0.0227   Alkaline phosphatase
    0.0134   Red blood cell (erythrocyte) distribution width
    0.0104   Albumin
    0.0079   Alanine aminotransferase
    0.0077   Estimated glomerular filtration rate, serum creatinine + cystain C
    0.0072   Other disorders of bladder
    0.0071   Aspartate aminotransferase
    0.0071   Other retinal disorders
    0.0065   Cystatin C
    0.0064   N32 Other disorders of bladder
    0.0062   Calcium
    0.0061   Gamma glutamyltransferase
    0.0057   Diabetic retinopathy
    0.0057   Hereditary retinal dystrophies
    0.0057   H36 Retinal disorders in diseases classified elsewhere



In [63]:
pan_ukb_pleiotropy(591)

<h2>Hypertension</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 2 (Contribution: 0.0982)</span>

    0.0595   Standing height
    0.0290   Sitting height
    0.0227   Seated height
    0.0197   Forced vital capacity (FVC)
    0.0193   Forced vital capacity (FVC), Best measure
    0.0169   Mean arterial pressure, combined automated + manual reading, adjusted by medication
    0.0163   Systolic blood pressure, combined automated + manual reading, adjusted by medication
    0.0157   Comparative height size at age 10
    0.0157   Forced expiratory volume in 1-second (FEV1)
    0.0156   Mean arterial pressure, automated reading, adjusted by medication
    0.0152   Forced expiratory volume in 1-second (FEV1), predicted
    0.0150   Forced expiratory volume in 1-second (FEV1), Best measure
    0.0149   Systolic blood pressure, automated reading, adjusted by medication
    0.0138   Trunk fat-free mass
    0.0138   Trunk predicted mass



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 15 (Contribution: 0.1904)</span>

    0.0670   Essential hypertension
    0.0596   Hypertension
    0.0278   E78 Disorders of lipoprotein metabolism and other lipidaemias
    0.0271   Disorders of lipoid metabolism
    0.0265   Hypercholesterolemia
    0.0256   Hyperlipidemia
    0.0252   Ischemic Heart Disease
    0.0197   Other chronic ischemic heart disease, unspecified
    0.0174   I25 Chronic ischaemic heart disease
    0.0138   antipyretic
    0.0138   paracetamol
    0.0134   opioid analgesic|antipyretic
    0.0130   Angina pectoris
    0.0111   E11 Non-insulin-dependent diabetes mellitus
    0.0100   Leg fat mass (left)



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 3 (Contribution: 0.1210)</span>

    0.0461   Systolic blood pressure, combined automated + manual reading, adjusted by medication
    0.0425   Systolic blood pressure, automated reading, adjusted by medication
    0.0405   Mean arterial pressure, combined automated + manual reading, adjusted by medication
    0.0373   Mean arterial pressure, automated reading, adjusted by medication
    0.0353   Systolic blood pressure, combined automated + manual reading
    0.0334   Systolic blood pressure, automated reading
    0.0304   Pulse pressure, combined automated + manual reading, adjusted by medication
    0.0288   Mean arterial pressure, combined automated + manual reading
    0.0282   Pulse pressure, automated reading, adjusted by medication
    0.0277   Diastolic blood pressure, combined automated + manual reading, adjusted by medication
    0.0274   Mean arterial pressure, automated reading
    0.0261   Diastolic blood pressure, automated reading, adjusted by medication
    0.0246   Pulse pressure, combined automated 

In [65]:
pan_ukb_pleiotropy(2350)

<h2>Standing Height</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 7 (Contribution: 0.2952)</span>

    0.0748   Impedance of whole body
    0.0676   Standing height
    0.0580   Impedance of arm (left)
    0.0572   Impedance of arm (right)
    0.0569   Impedance of leg (left)
    0.0559   Impedance of leg (right)
    0.0233   Sitting height
    0.0217   Seated height
    0.0181   Comparative height size at age 10
    0.0144   Forced expiratory volume in 1-second (FEV1), predicted
    0.0132   Trunk fat percentage
    0.0101   Body mass index (BMI)
    0.0098   Body mass index (BMI)
    0.0095   Reticulocyte count
    0.0094   Heel quantitative ultrasound index (QUI), direct entry (right)



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 4 (Contribution: 0.1740)</span>

    0.1274   Standing height
    0.0721   Sitting height
    0.0263   Seated height
    0.0192   Comparative height size at age 10
    0.0190   Trunk fat-free mass
    0.0189   Trunk predicted mass
    0.0134   Whole body fat-free mass
    0.0117   Whole body water mass
    0.0114   Waist circumference
    0.0112   Leg fat mass (left)
    0.0112   Arm fat mass (right)
    0.0107   Leg fat mass (right)
    0.0101   Arm fat mass (left)
    0.0099   E66 Obesity
    0.0099   Obesity



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 2 (Contribution: 0.2623)</span>

    0.0640   Standing height
    0.0351   Sitting height
    0.0250   Seated height
    0.0206   Forced vital capacity (FVC)
    0.0197   Forced vital capacity (FVC), Best measure
    0.0186   Comparative height size at age 10
    0.0170   Forced expiratory volume in 1-second (FEV1), predicted
    0.0161   Forced expiratory volume in 1-second (FEV1)
    0.0152   Forced expiratory volume in 1-second (FEV1), Best measure
    0.0132   Mean arterial pressure, combined automated + manual reading, adjusted by medication
    0.0126   Systolic blood pressure, combined automated + manual reading, adjusted by medication
    0.0121   Mean arterial pressure, automated reading, adjusted by medication
    0.0120   Leg fat percentage (left)
    0.0120   Leg fat percentage (right)
    0.0118   Trunk fat-free mass



In [69]:
pan_ukb_pleiotropy(1747)

<h2>Intake Of Sugar Added To Coffee</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 52 (Contribution: 0.1127)</span>

    0.0152   Saturated fat
    0.0138   Albumin
    0.0124   Fat
    0.0108   Osteoporosis
    0.0108   Osteoporosis NOS
    0.0093   Vitamin C
    0.0086   Home location - north co-ordinate (rounded)
    0.0084   methotrexate
    0.0084   anti-metabolite
    0.0081   Retinol
    0.0079   Home location at assessment - north co-ordinate (rounded)
    0.0072   Carotene
    0.0071   Osteoporosis, osteopenia and pathological fracture
    0.0069   xanthine oxidase inhibitor|anti-gout agent
    0.0069   allopurinol



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 54 (Contribution: 0.0723)</span>

    0.0427   Whole body fat-free mass
    0.0224   NSAID|non-steroidal anti-inflammatory drug
    0.0193   Leg fat mass (right)
    0.0167   Usual walking pace
    0.0119   Z86 Personal history of certain other diseases
    0.0118   Weight
    0.0100   clarithromycin
    0.0099   Arm predicted mass (left)
    0.0085   Whole body fat mass
    0.0071   Pulse pressure, combined automated + manual reading, adjusted by medication
    0.0066   Leg fat mass (left)
    0.0065   lansoprazole
    0.0063   aspirin
    0.0063   diclofenac
    0.0061   Other mental disorder



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 75 (Contribution: 0.0839)</span>

    0.0341   SHBG
    0.0298   Pulse rate, automated reading
    0.0129   Pulse rate
    0.0122   Testosterone
    0.0118   COVID-19 positive (controls include untested)
    0.0118   COVID-19 positive (controls include untested), only patients from centers in England
    0.0113   COVID-19 positive (controls include untested)
    0.0112   COVID-19 positive (controls include untested), only patients from centers in England
    0.0099   COVID-19 positive (controls only COVID-19 negative)
    0.0097   Estimated glomerular filtration rate, serum creatinine + cystain C
    0.0096   COVID-19 positive (controls only COVID-19 negative)
    0.0090   Cystatin C
    0.0090   Fasciitis
    0.0089   Contracture of palmar fascia [Dupuytren's disease]
    0.0088   Disorders of muscle, ligament, and fascia



In [70]:
pan_ukb_pleiotropy(1677)

<h2>Covid-19 Positive (Controls Only Covid-19 Negative)</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 42 (Contribution: 0.4360)</span>

    0.0801   COVID-19 positive (controls include untested)
    0.0800   COVID-19 positive (controls include untested), only patients from centers in England
    0.0792   COVID-19 positive (controls include untested)
    0.0791   COVID-19 positive (controls include untested), only patients from centers in England
    0.0557   COVID-19 positive (controls only COVID-19 negative)
    0.0498   COVID-19 positive (controls only COVID-19 negative)
    0.0135   Cholelithiasis
    0.0132   Cholelithiasis and cholecystitis
    0.0130   K80 Cholelithiasis
    0.0094   Chronic dermatitis due to solar radiation
    0.0093   L57 Skin changes due to chronic exposure to nonionising radiation
    0.0082   Dermatitis due to solar radiation
    0.0069   Actinic keratosis
    0.0068   Breast cancer [female]
    0.0066   Breast cancer



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 47 (Contribution: 0.0847)</span>

    0.0245   Age first had sexual intercourse
    0.0170   Forced expiratory volume in 1-second (FEV1), predicted
    0.0135   ACE inhibitor|anti-hypertensive
    0.0126   beta-lactam antibiotic|antibiotic
    0.0115   Forced expiratory volume in 1-second (FEV1), Best measure
    0.0115   Whole body fat-free mass
    0.0112   Arm fat-free mass (left)
    0.0096   Trunk fat percentage
    0.0093   Waist circumference
    0.0087   Leg predicted mass (left)
    0.0085   Impedance of leg (right)
    0.0081   Leg fat-free mass (left)
    0.0070   Ankle spacing width
    0.0069   Body fat percentage
    0.0059   Cholesterol



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 67 (Contribution: 0.5227)</span>

    0.0846   COVID-19 positive (controls include untested), only patients from centers in England
    0.0846   COVID-19 positive (controls include untested)
    0.0844   COVID-19 positive (controls include untested), only patients from centers in England
    0.0843   COVID-19 positive (controls include untested)
    0.0765   COVID-19 positive (controls only COVID-19 negative)
    0.0720   COVID-19 positive (controls only COVID-19 negative)
    0.0317   IGF-1
    0.0114   Platelet distribution width
    0.0088   Estimated glomerular filtration rate, serum creatinine + cystain C
    0.0069   Cystatin C
    0.0047   Systolic blood pressure, manual reading
    0.0044   Systolic blood pressure, manual reading, adjusted by medication
    0.0040   Mean arterial pressure, manual reading
    0.0038   Estimated glomerular filtration rate, serum creatinine
    0.0037   Creatinine



# Neuropsychiatric Disorders

In [71]:
for z in range(52,58):
    pan_ukb_pleiotropy(z)

<h2>F05 Delirium, Not Induced By Alcohol And Other Psychoactive Substances</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 99 (Contribution: 0.1305)</span>

    0.0148   Cardiac arrest
    0.0146   Cardiac arrest and ventricular fibrillation
    0.0144   I46 Cardiac arrest
    0.0130   Other disorders of pancreatic internal secretion
    0.0125   Hypoglycemia
    0.0122   E16 Other disorders of pancreatic internal secretion
    0.0112   Obesity
    0.0111   E66 Obesity
    0.0104   E21 Hyperparathyroidism and other disorders of parathyroid gland
    0.0104   G47 Sleep disorders
    0.0103   Hyperparathyroidism
    0.0101   Sleep disorders
    0.0101   Overweight, obesity and other hyperalimentation
    0.0090   Varicose veins of lower extremity
    0.0088   Disorders of parathyroid gland



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

...

<span style='color:None'>Top Factor: 93 (Contribution: 0.0847)</span>

    0.0244   Impedance of arm (right)
    0.0191   Trunk fat mass
    0.0179   E11 Non-insulin-dependent diabetes mellitus
    0.0148   Impedance of leg (right)
    0.0123   Type 2 diabetes
    0.0104   Leg fat-free mass (left)
    0.0102   Leg fat mass (left)
    0.0100   Leg fat mass (right)
    0.0100   doxycycline
    0.0100   Whole body water mass
    0.0090   Body mass index (BMI)
    0.0087   Trunk fat percentage
    0.0086   tetracycline antibiotic|antibiotic
    0.0081   ramipril
    0.0077   Cystatin C



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 53 (Contribution: 0.0682)</span>

    0.0102   Corneal resistance factor (right)
    0.0096   X61 Intentional self-poisoning by and exposure to antiepileptic, sedative-hypnotic, anti-Parkinsonism and psychotropic drugs, not elsewhere classified
    0.0091   Corneal resistance factor (left)
    0.0090   Poisoning by psychotropic agents
    0.0085   SHBG
    0.0083   T43 Poisoning by psychotropic drugs, not elsewhere classified
    0.0083   Suicide or self-inflicted injury
    0.0080   Suicidal ideation or attempt
    0.0075   Home location - north co-ordinate (rounded)
    0.0075   Home location at assessment - north co-ordinate (rounded)
    0.0074   Toxic effect of (non-ethyl) alcohol and petroleum and other solvents
    0.0073   T51 Toxic effect of alcohol
    0.0073   T39 Poisoning by nonopioid analgesics, antipyretics and antirheumatics
    0.0073   Corneal hysteresis (right)
    0.0072   X60 Intentional self-poisoning by and exposure to nonopioid analgesics, antipyretics and antirheumatics



In [73]:
for z in range(520,538):
    pan_ukb_pleiotropy(z)

<h2>Other Mental Disorder</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 6 (Contribution: 0.1623)</span>

    0.0290   Trunk fat percentage
    0.0231   Body fat percentage
    0.0229   Impedance of whole body
    0.0226   Impedance of arm (right)
    0.0224   Impedance of arm (left)
    0.0194   Trunk fat mass
    0.0155   Arm fat percentage (left)
    0.0154   Whole body fat mass
    0.0153   Arm fat percentage (right)
    0.0139   Leg fat percentage (right)
    0.0133   Leg fat percentage (left)
    0.0132   Impedance of leg (left)
    0.0129   Impedance of leg (right)
    0.0090   Leg fat mass (right)
    0.0088   Arm fat mass (left)



<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

...

<span style='color:None'>Top Factor: 96 (Contribution: 0.1300)</span>

    0.0249   Ankle spacing width (right)
    0.0187   Impedance of leg (left)
    0.0154   proton pump inhibitor|PPI
    0.0153   Angina pectoris
    0.0143   Whole body fat-free mass
    0.0134   Non-cancer illness code, self-reported
    0.0127   benzodiazepine|sedative|anxiety
    0.0115   Ischemic Heart Disease
    0.0111   Sitting height
    0.0088   Leg predicted mass (left)
    0.0088   Seated height
    0.0085   Leg fat mass (left)
    0.0084   Peak expiratory flow (PEF)
    0.0072   Arm predicted mass (right)
    0.0070   Age at first live birth



<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 61 (Contribution: 0.0849)</span>

    0.0324   Heel bone mineral density (BMD)
    0.0322   Heel bone mineral density (BMD) T-score, automated
    0.0322   Heel quantitative ultrasound index (QUI), direct entry
    0.0258   Heel Broadband ultrasound attenuation, direct entry
    0.0139   Urate
    0.0119   Heel bone mineral density (BMD) T-score, automated (left)
    0.0119   Heel quantitative ultrasound index (QUI), direct entry (left)
    0.0119   Heel broadband ultrasound attenuation (left)
    0.0116   Heel bone mineral density (BMD) (left)
    0.0115   Heel bone mineral density (BMD) T-score, automated (right)
    0.0115   Heel quantitative ultrasound index (QUI), direct entry (right)
    0.0115   Heel broadband ultrasound attenuation (right)
    0.0114   Heel bone mineral density (BMD) (right)
    0.0103   Cerebrovascular disease
    0.0102   Creatinine

